In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
import matplotlib.pyplot as plt

import tensorflow as tf

In [ ]:
!ls -lrth /kaggle/input/plant-seedlings-classification

In [ ]:
!ls -lrth /kaggle/input/plant-seedlings-classification/train/

In [ ]:
image_size = 256
batch_size = 8

In [ ]:
image_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.2
)

In [ ]:
#Build training generator. 
train_generator = image_datagenerator.flow_from_directory('/kaggle/input/plant-seedlings-classification/train/',
                                                    target_size=(image_size, image_size),
                                                    subset='training',
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True,
                                                    seed= 9
                                                )

#Build validation generator
val_generator = image_datagenerator.flow_from_directory('/kaggle/input/plant-seedlings-classification/train/',
                                                   target_size=(image_size, image_size),                                                   
                                                   subset='validation',
                                                   batch_size=batch_size,
                                                   class_mode='categorical',
                                                   shuffle=True,
                                                   seed= 9
                                                )

In [ ]:
train_generator.class_indices

In [ ]:
class_mapping = {
0:"Black-grass",
1:"Charlock",
2:"Cleavers",
3:"Common-Chickweed",
4:"Common-wheat",
5:"Fat-Hen",
6:"Loose-Silky-bent",
7:"Maize",
8:"Scentless-Mayweed",
9:"Shepherds-Purse",
10:"Small-flowered-Cranesbill",
11:"Sugar-beet",
}

In [ ]:
#Clear any previous model from memory
tf.keras.backend.clear_session()

#Initialize model
model = tf.keras.models.Sequential()

#normalize data
model.add(tf.keras.layers.InputLayer(input_shape=(image_size,image_size,3,)))

#Add Conv Layer
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3),strides = (2,2), activation='relu'))

#normalize data
model.add(tf.keras.layers.BatchNormalization())

#Add Conv Layer
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), strides = (1,1), activation='relu'))

#Add Max Pool layer
model.add(tf.keras.layers.MaxPool2D(pool_size = (2,2)))

#normalize data
model.add(tf.keras.layers.BatchNormalization())

#Add Conv Layer
model.add(tf.keras.layers.Conv2D(128, kernel_size=(3,3), strides = (1,1), activation='relu'))

#Add Max Pool layer
model.add(tf.keras.layers.MaxPool2D(pool_size = (3,3)))

#normalize data
model.add(tf.keras.layers.BatchNormalization())

#Add Max Pool layer
model.add(tf.keras.layers.GlobalMaxPool2D())

#Add Dense Layers after flattening the data
model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(256, activation='relu'))

#Add Dropout
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(256, activation='relu'))

#Add Dropout
model.add(tf.keras.layers.Dropout(0.4))

#Add Output Layer
model.add(tf.keras.layers.Dense(12, activation='softmax'))

In [ ]:
#Specify Loass and Optimizer
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#Model Summary
model.summary()

In [ ]:
#Saving the best model using model checkpoint callback
model_checkpoint=tf.keras.callbacks.ModelCheckpoint('crop_vs_grass.h5', #where to save the model
                                                    save_best_only=True, 
                                                    monitor='val_accuracy', 
                                                    mode='max', 
                                                    verbose = 2)

In [ ]:
history = model.fit(train_generator,
          epochs=200,
          steps_per_epoch= 3803//batch_size,  #Number of training images//batch_size
          validation_data=val_generator,
          validation_steps = 947//batch_size, #Number of validation images//batch_size
          callbacks = [model_checkpoint],
          verbose = 2
)

In [ ]:
history_df = pd.DataFrame(history.history)

In [ ]:
history_df.loc[5:,['loss','val_loss']].plot()

In [ ]:
history_df.loc[5:,['accuracy','val_accuracy']].plot()

In [ ]:
STEP_SIZE_VAL = val_generator.n // val_generator.batch_size
model.evaluate_generator(generator=val_generator, steps=STEP_SIZE_VAL)

In [ ]:
def create_test_dataset(test_images_dir, image_size, rescale = 1):
    filenames = []
    test_data = []
    
    for img_file in os.listdir(test_images_dir):
        filenames.append(img_file)

        img = tf.keras.preprocessing.image.load_img(test_images_dir + '/' + img_file)
        img_new = img.resize([image_size, image_size])
        img_array = tf.keras.preprocessing.image.img_to_array(img_new)
        if rescale is not 1:
            img_array = img_array * 1./255
        test_data.append(img_array)
    
    test_data = np.array(test_data,dtype=np.float32)
    
    return test_data, filenames 

In [ ]:
test_data, filenames  = create_test_dataset('/kaggle/input/plant-seedlings-classification/test/',image_size, rescale = 1./255)

In [ ]:
test_data[0].shape

In [ ]:
predicted_class = []
for img in test_data:
    img_instance = np.expand_dims(img, axis=0)
    img_instance_class = model.predict(img_instance)
    img_instance_class = np.argmax(img_instance_class,axis=1)
    img_instance_class = class_mapping.get(img_instance_class[0], np.nan)
    predicted_class.append(img_instance_class)
    pass

In [ ]:
results = pd.DataFrame({"file":filenames,"species":predicted_class})
results.to_csv('submission_fimal.csv', index = False)
results.head()

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("plant_seedling.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights("plant_seedling.h5")

print("Saved model to disk")